### Užduotis

Pirma užduotis reikalaus realizuoti:

- [X] efektyvią duomenų nuskaitymo programą su pasirinku egzistuojančiu iš anksto apmokytu (angl. pre-trained) vaizdų klasifikavimo modeliu,
- [X] paskaičiuoti tikslumo, precizijos, atkūrimo ir F1 statistikas pasirinktiems 1000 paveikslėlių iš OpenImages,
- [X] realizuoti slenkstinės reikšmės (angl. threshold) keitimą, įgalinant klasifikuoti vaizdus kiekvienai užduotai klasei keičiant . Statistikos turi persiskaičiuoti po slenkstinės reikšmės pakeitimo.

In [1]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [2]:
from pathlib import Path
from typing import Callable
from torch import Tensor
from torch.utils.data import Dataset
from PIL import Image
from importlib import reload
from copy import deepcopy
from lab1.mydataset import MyDataSet
import lab1

In [3]:
from lab1.device import device
reload(lab1.device)
device

device(type='mps')

In [4]:
from lab1.download import download_if_not_found
reload(lab1.download)
download_if_not_found()

In [5]:
from torchvision.models import resnet101, ResNet101_Weights

weights: ResNet101_Weights = ResNet101_Weights.DEFAULT

model = resnet101(weights=weights, progress=True).to(device).eval()

catmap = dict([(i, c) for c, i in enumerate(weights.meta['categories'])])

In [6]:
from lab1.trans import img_trans_1

img_trans_2 = weights.transforms()
dataset = MyDataSet("data", preprocess_fn = img_trans_1, label_map = catmap)

In [7]:
from torch.utils.data import DataLoader

data_loader = DataLoader(dataset, batch_size = 2**4, num_workers=0)

In [8]:
from functools import reduce
import numpy as np
import torch
from torch import cat
from torch.utils.data import Dataset
from torchvision.transforms import Compose, Resize, ToTensor, Normalize

predictions = torch.empty((0, 1000))
truths_ = torch.tensor([], dtype=int)

for images, ts in data_loader:
    truths_  = torch.cat((truths_, ts))
    # Can be though as list of ŷ
    #     ŷ_i - confidence that input image belongs to i-th class
    #              "squishes" outputs to a range of [0, 1] 👇
    from_model = model(images.to(device)).detach().cpu().sigmoid()
    predictions = cat((predictions, from_model))

# Initializes list of vectors:
#   truths - 
#   y - ground truth vector, with only one (i-th element), representing a class set to true
truths = torch.zeros_like(predictions, dtype=torch.bool)
for index, label in enumerate(truths_):
    truths[index,label.item()] = True

truths, predictions

(tensor([[False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         ...,
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False]]),
 tensor([[0.4655, 0.5691, 0.4720,  ..., 0.4166, 0.3569, 0.4560],
         [0.4851, 0.5180, 0.5254,  ..., 0.4793, 0.5382, 0.5099],
         [0.4731, 0.4663, 0.4910,  ..., 0.4731, 0.5712, 0.5123],
         ...,
         [0.4022, 0.6038, 0.3598,  ..., 0.4179, 0.5165, 0.6042],
         [0.4610, 0.4933, 0.4728,  ..., 0.4774, 0.3940, 0.5322],
         [0.4491, 0.4607, 0.5754,  ..., 0.5441, 0.3628, 0.4592]]))

In [9]:
from lab1.stat import table, stats
def test():
    tp, tn, fp, fn = table(truths, predictions)
    return tp + tn + fp + fn == truths_.numel()

table(truths, predictions), test()


((tensor(0), tensor(539), tensor(477), tensor(0)), tensor(True))

In [10]:
from lab1.download import classes

for label in classes:
    print(f"---------------------")
    print(f"Class: {label}")
    acc, re, pre, f1 = stats(table(truths, predictions, labels=catmap[label.lower()], threshold = 0.8))
    print(f" . Accuracy:  {acc:.3}\n . Recall:    {re:.3}\n . Precision: {pre:.3}\n . F1:        {f1:.3}\n")

---------------------
Class: Pizza
 . Accuracy:  0.997
 . Recall:    0.997
 . Precision: 0.994
 . F1:        0.996

---------------------
Class: Television
 . Accuracy:  0.981
 . Recall:    0.968
 . Precision: 0.976
 . F1:        0.972

---------------------
Class: Tiger
 . Accuracy:  0.98
 . Recall:    0.922
 . Precision: 1.0
 . F1:        0.959

---------------------
Class: Snowplow
 . Accuracy:  0.997
 . Recall:    1.0
 . Precision: 0.964
 . F1:        0.982

